# calculate integrated grid

In [1]:
import numpy as np
from astropy.io import fits
from speedysedfit.integrate_grid import calc_integrated_grid, get_responses, calc_comarcs_integrated_grid
from speedysedfit import model
from speedysedfit import model, reddening, filters
from astropy.io import ascii, fits
from astropy.table import Table
from multiprocessing import cpu_count, Manager, Process 

# get the SPEEDYFIT_MODELS file

In [2]:
import os
import glob
basedir=os.environ.get('SPEEDYFIT_MODELS', None)
dire = os.path.join(basedir, '..', 'transmission_curves', '*SWIFT*')
print(dire)
glob.glob(dire)

/home/lijiao/lijiao/speedyfit/modelgrids/../transmission_curves/*SWIFT*


['/home/lijiao/lijiao/speedyfit/modelgrids/../transmission_curves/SWIFT.white',
 '/home/lijiao/lijiao/speedyfit/modelgrids/../transmission_curves/SWIFT.UVM2',
 '/home/lijiao/lijiao/speedyfit/modelgrids/../transmission_curves/SWIFT.UVW1',
 '/home/lijiao/lijiao/speedyfit/modelgrids/../transmission_curves/SWIFT.U',
 '/home/lijiao/lijiao/speedyfit/modelgrids/../transmission_curves/SWIFT.B',
 '/home/lijiao/lijiao/speedyfit/modelgrids/../transmission_curves/SWIFT.V',
 '/home/lijiao/lijiao/speedyfit/modelgrids/../transmission_curves/SWIFT.UVW2']

In [3]:
ebvs=np.r_[0:2.1:0.05]

In [4]:
gridfile = model.get_grid_file(integrated=False, grid=os.path.join(basedir, 'TlustyO_G_v10.fits'))
print(gridfile)

/home/lijiao/lijiao/speedyfit/modelgrids/TlustyO_G_v10.fits


In [5]:
gridfile = model.get_grid_file(integrated=False, grid='kurucz')
print(gridfile)

/home/lijiao/lijiao/speedyfit/modelgrids/kurucz93_z0.0_k2odfnew_sed.fits


In [6]:
gridfile = model.get_grid_file(integrated=False, grid='comarcs')
print(gridfile)

/home/lijiao/lijiao/speedyfit/modelgrids/COMARCS.fits


In [7]:
responses0 = ['SWIFT', 'GALEX', 'IUE', 'STROMGREN', 'JOHNSON', 'GAIA3E', 'GAIA2', 'SKYMAPPER', 'APASS', 'SDSS', '2MASS', 'WISE', 'PANSTARRS']

In [8]:
filters.list_response(name='SWIFT')

['SWIFT.B',
 'SWIFT.U',
 'SWIFT.UVM2',
 'SWIFT.UVW1',
 'SWIFT.UVW2',
 'SWIFT.V',
 'SWIFT.white']

In [9]:
filters.eff_wave('SWIFT.UVW2')

2054.61

In [10]:
filters.list_response('SWIFT')

['SWIFT.B',
 'SWIFT.U',
 'SWIFT.UVM2',
 'SWIFT.UVW1',
 'SWIFT.UVW2',
 'SWIFT.V',
 'SWIFT.white']

In [ ]:
ff = fits.open(gridfile)
data = ff[1].data
wave, flux = data['wavelength'], data['flux']
teffs = []
loggs = []
hdus = []
for hdu in ff[1:]:
    teffs.append(float(hdu.header['TEFF']))
    loggs.append(float(hdu.header['LOGG']))
    hdus.append(hdu)
responses = get_responses(responses=responses0, wave=wave)
# -- definition of one process for multi processing:

law='fitzpatrick2004'
Rv=3.1
grid='kurucz'
    
def do_ebv_process(ebvs, arr, responses):
    # Run over all reddening values, calculate the reddened model and integrate it over all photbands.
    for ebv in ebvs:
        # redden the model
        flux_ = reddening.redden(flux, wave=wave, ebv=ebv, rtype='flux', law=law, Rv=Rv)
        # calculate synthetic fluxes
        synflux = filters.synthetic_flux(wave, flux_, responses)
        # append to results
        arr.append([np.concatenate(([ebv], synflux))])
        
# -- prepare the array containing the integrated fluxes
#   (1 row per model, 1 column for each response curve and teff, logg, ebv and total luminosity)
output = np.zeros((len(teffs) * len(ebvs), 4 + len(responses)))
start = 0

manager = Manager()                                                                                                                                               
arr = manager.list([])

In [15]:
do_ebv_process(ebvs[0:1], arr, responses[0:2])

In [13]:
filters.list_response('PANSTARRS')

['PANSTARRS.G', 'PANSTARRS.I', 'PANSTARRS.R', 'PANSTARRS.Y', 'PANSTARRS.Z']

# calculate integrated grid of kurucz

In [6]:
calc_integrated_grid(threads=15, ebvs=ebvs, law='fitzpatrick2004', Rv=3.1,
                         responses=responses0, grid='kurucz') 

In [19]:
!mv ikurucz93_z0.0_k2odfnew_sed_lawfitzpatrick2004_Rv3.10.fits $SPEEDYFIT_MODELS

/home/lijiao/lijiao/speedyfit/modelgrids/TlustyO_G_v10.fits


# calculate integrated grid of tmap

In [23]:
#calc_integrated_grid(threads=15, ebvs=ebvs, law='fitzpatrick2004', Rv=3.1,
#                         responses=responses0, grid='tmap') 

In [22]:
#!mv iTMAP2012_sdOB_extended_lawfitzpatrick2004_Rv3.10.fits $SPEEDYFIT_MODELS

# calculate integrated grid of munari

In [3]:
#calc_integrated_grid(threads=15, ebvs=ebvs, law='fitzpatrick2004', Rv=3.1,
#                         responses=responses0, grid='munari') 

In [2]:
!mv iMunari2005_extended_lawfitzpatrick2004_Rv3.10.fits $SPEEDYFIT_MODELS

# calculate intergrated grid of TLUSTY
- B star

!mv iTlustyB_G_v2_lawfitzpatrick2004_Rv3.10.fits $SPEEDYFIT_MODELS

- O star

In [21]:
gridfile = model.get_grid_file(integrated=True, grid='tlustyO')
print(gridfile)

/home/lijiao/lijiao/speedyfit/modelgrids/iTlustyO_G_v10_lawfitzpatrick2004_Rv3.10.fits


In [26]:
!mv iTlustyO_G_v10_lawfitzpatrick2004_Rv3.10.fits $SPEEDYFIT_MODELS

# calculate integrated grid of COMARCS

In [8]:
#calc_comarcs_integrated_grid(threads=30, ebvs=ebvs, law='fitzpatrick2004', Rv=3.1,
#                         responses=responses0, grid='comarcs') 

In [10]:
!mv iCOMARCS_lawfitzpatrick2004_Rv3.10.fits $SPEEDYFIT_MODELS

check integrated grid of COMARCS

In [9]:
#hdul = fits.open(os.path.join(basedir, 'iCOMARCS_lawfitzpatrick2004_Rv3.10.fits'))

In [ ]:
tab = Table(hdul[1].data)
tab